In [2]:
from tokenizer import TikToken200k
from model import *

AttributeError: 'Encoding' object has no attribute 'vocab'

In [3]:
phrase = 'Abraham Lincoln'
ids = tokenizer.encode(phrase)

emb = encoding_layer(torch.tensor(ids).reshape(-1, 1))
attn = stacked_attention(emb)

In [4]:
attn[1].shape

torch.Size([2, 12, 1, 1])

In [5]:
ids

[53339, 50439]